In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
import tensorflow as tf
from tensorflow.keras import layers, models
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input director

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-02-22 19:57:40.426895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 19:57:40.427072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 19:57:40.606918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Introduction

La pneumonie est une maladie du a une infections des poumons. les agents pathogène comme virus, les bactéries, des champignons ou d'autres micro-organismes en sont les princiaples cause. Elle se caractérise par un remplessement des sacs d'aires des poumons appelé alvéoles, qui peuvent se remplir de liquide, rendant ainsi la respiration difficile.
D'après l'OMS, la pneumonie est la première cause infectieuse de mortalité chez l’enfant à l’échelle mondiale. En 2019, 740 180 enfants de moins de 5 ans sont décédés des suites d’une pneumonie, soit 14 % de tous les décès survenus dans cette tranche d’âge ; chez les enfants de 1 à 5 ans, la pneumonie est responsable de 22 % des décès. 

Les fichiers de ces images sont sur kaggle, nous allons les accédés a travers leurs chement. Les fichiers sont organiser en trois dossier, qui sont test, train, val respectivement pour les données de test, d'entrainement et de validation de notre modèle. Nous allons commencer dans un premier temps á définir des variables correspondants a ces dossiers, puis après construire les dataframe correspondant en transfomant les fichiers images en données numerique.
Dans chaque dossier, se trouve deux sous-dossier, correspondant aux images normal et ceux ayant la pneunomie. On va parcouris les fichiers de chaque sous dossier, puis transformer le chifier image en tableau numerique a attribué une colonne qui peux prendre la valeur 0 ou 1 selon si l'image est normal ou anormal. Etand données que nous allons effectué cette même operation sur les trois dossier (test, train, et val), ce serait mieux de créer une fonction et la faire appèle en passant en paramètre le dossier correspondant.

In [2]:
test_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test/"
train_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/"
val_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/"
def preparation(dossier): 
        X, y = [], []
        for i in ["NORMAL", "PNEUMONIA"]:
            chemin = dossier + i
            for file in os.listdir(chemin):
                chemin_fichier = os.path.join(chemin, file)
                if not chemin_fichier.endswith(".DS_Store"):
                    image = cv2.imread(chemin_fichier)
                    image = cv2.resize(image, (100, 100))
                    X.append(np.array(image))
                    if i=="NORMAL":
                        y.append(0)
                    else:
                        y.append(1)
        return np.array(X), np.array(y)
X_val, y_val = preparation(val_dir)
X_test, y_test = preparation(test_dir)
X_train, y_train = preparation(train_dir)

## Modèle utilisé

Le model,qui est utilisé pour détècté la presence ou non de pneumonie est le CNN. C'est un réseau de neuronne qui fait ces preuve en vision par ordinateur dans divers domains. Il est le mieux adapté pour notre problème. L'objectif de modèle CNN est d'extraire les caractéristique de l'object en question en intégrant la notion de non linéareté, la ou á échoué la pluspart des modèles traditionnelle de machine learning.
La mise en place de notre modèle est fait en utilisant la technique sequentielle de trensorflow. Pour rappel il y a deux manière de mètre en place les modèles sur le plateforme de trensorflow, la manière sequentielle et fonctionnelle.
Nous utilisont un modèle d'un seul couche, on commence par ajouté la couche convolutionnaire avec un filtre de dimension (3,3) et pour fonction d'activation la fonction 'relu'. il faut aussi noté que notre image a trois cannaux correspondant au different cannaux de RGB, raison pour laquelle la dimension de notre image est de (100, 100, 3 )
Nous ajoutons ensuite, le Pooling consistant á reduire la dimension spatiale du resultat obtenue dans la couche convolutionnaire. Nous avons utilisé le Pooling maximum avec une fenêtre de (2, 2). 
Le resultat est enseuite vertorisé,

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
history = model.fit(X_train, y_train, epochs=10)

Epoch 1/10
163/163 [==============================] - 24s 140ms/step - loss: 31.5249 - accuracy: 0.8779
Epoch 2/10
163/163 [==============================] - 23s 140ms/step - loss: 0.9248 - accuracy: 0.9615
Epoch 3/10
163/163 [==============================] - 23s 140ms/step - loss: 0.2954 - accuracy: 0.9774
Epoch 4/10
163/163 [==============================] - 23s 140ms/step - loss: 0.1143 - accuracy: 0.9875
Epoch 5/10
163/163 [==============================] - 23s 138ms/step - loss: 0.0647 - accuracy: 0.9921
Epoch 6/10
163/163 [==============================] - 23s 141ms/step - loss: 0.0588 - accuracy: 0.9923
Epoch 7/10
163/163 [==============================] - 23s 140ms/step - loss: 0.1254 - accuracy: 0.9873
Epoch 8/10
163/163 [==============================] - 22s 136ms/step - loss: 0.1853 - accuracy: 0.9864
Epoch 9/10
163/163 [==============================] - 22s 136ms/step - loss: 0.1030 - accuracy: 0.9902
Epoch 10/10
163/163 [==============================] - 23s 139ms/step - 